In [ ]:
import bw2io
from pathlib import Path
import bw2data
from brightpath import SimaproConverter
fp="ESA db/ESA_LCA_DB_1.2.0f_e3.9.1_-_EXTERNAL_csv_original.CSV"
bw2data.projects.set_current("ecoinvent-3.9.1-cutoff")
c = SimaproConverter(fp, ecoinvent_version="3.9")

In [ ]:
c.convert_to_brightway()

In [ ]:
# if links to EI 35
#c.i.data = bw2io.strategies.migrate_exchanges(
#    db=c.i.data,
#    migration="ESA-ei-3.5"
#)

In [ ]:
c.i.match_database(fields=["name", "reference product", "location"])
c.i.match_database("ecoinvent-3.9.1-cutoff", fields=["name", "reference product", "location"])
c.i.match_database("ecoinvent-3.9.1-biosphere", fields=["name", "categories"])

In [ ]:
c.i.statistics()

In [ ]:
c.i.drop_unlinked(i_am_reckless=True)
if c.i.db_name in bw2data.databases:
    del bw2data.databases[c.i.db_name]
c.i.write_database()

In [ ]:
# this is to build a migration file
migration = {
    "fields": ["name", "reference product", "location", ],
}

migration["data"] = [
    (
        (u["name"], u["reference product"], u["location"], ),
        {}
    )
    for u in c.i.unlinked
    if u["type"] == "technosphere"
]

In [ ]:
import json
with open("ESA-ei-3.5_migration.json", 'w') as fp:
    json.dump(migration, fp, indent=2)

In [ ]:
with open("ESA-ei-3.5_migration.json") as f_in:
    migration = json.load(f_in)

In [ ]:
import pandas as pd
pd.DataFrame.from_dict(c.i.unlinked)[["name", "reference product", "location", "categories", "unit", "simapro name", "type"]].to_excel("unlinked_exchanges.xlsx")

In [ ]:
import bw2io
from pathlib import Path
import bw2data
from brightpath import SimaproConverter
fp="lci-ESA-LCA-External-120f-ei391.xlsx"
bw2data.projects.set_current("ecoinvent-3.10-cutoff")

In [ ]:
i = bw2io.ExcelImporter(fp)

In [ ]:
i.apply_strategies()

In [ ]:
i.match_database(fields=["name", "reference product", "location"])
i.match_database("ecoinvent-3.10-cutoff", fields=["name", "reference product", "location"])
i.match_database("ecoinvent-3.10-biosphere", fields=["name", "categories"])

In [ ]:
i.statistics()

In [ ]:
import json
with open("ecoinvent-3.9.1-cutoff-ecoinvent-3.10-cutoff.json") as f_in:
    migration = json.load(f_in)

In [ ]:
migration = {
    (
        a["source"]["name"],
        a["source"]["reference product"],
        a["source"]["location"],
        #a["source"]["unit"]
    ): a["target"]
    for a in migration["replace"]
}

In [ ]:
for act in i.data:
    for exc in act["exchanges"]:
        if exc["type"] == "technosphere":
            key = (
                exc["name"],
                exc["reference product"],
                exc["location"],
                #exc["unit"]
            )
            if key in migration:
                exc["name"] = migration[key]["name"]
                exc["reference product"] = migration[key]["reference product"]
                #exc["unit"] = migration[key]["unit"]
                exc["location"] = migration[key]["location"]

In [ ]:
i.match_database(fields=["name", "reference product", "location"])
i.match_database("ecoinvent-3.10-cutoff", fields=["name", "reference product", "location"])
i.match_database("ecoinvent-3.10-biosphere", fields=["name", "categories"])
i.statistics()

In [ ]:
import json
with open("ecoinvent-3.9.1-cutoff-ecoinvent-3.10-cutoff.json") as f_in:
    migration = json.load(f_in)

In [ ]:
migration = {
    (
        a["source"]["name"],
        a["source"]["reference product"],
        a["source"]["location"],
        #a["source"]["unit"]
    ): a["targets"]
    for a in migration["disaggregate"]
}
len(migration)

In [ ]:
for act in i.data:
    new_excs = []
    for exc in act["exchanges"]:
        if exc["type"] == "technosphere":
            key = (
                exc["name"],
                exc["reference product"],
                exc["location"],
                #exc["unit"]
            )
            if key in migration:
                for target in migration[key]:
                    new_excs.append(
                        {
                            "name": target["name"],
                            "reference product": target["reference product"],
                            "unit": exc["unit"],
                            "location": target["location"],
                            "uncertainty type": exc.get("uncertainty type", 0),
                            "loc": exc.get("loc", 0),
                            "scale": exc.get("scale", 0),
                            "amount": exc["amount"] * target["allocation"],
                            "type": "technosphere"
                        }
                    )
                exc["delete"] = True
    if len(new_excs) > 0:
        act["exchanges"].extend(new_excs)
    act["exchanges"] = [e for e in act["exchanges"] if e.get("delete", False) is False]
                

In [ ]:
i.match_database(fields=["name", "reference product", "location"])
i.match_database("ecoinvent-3.10-cutoff", fields=["name", "reference product", "location"])
i.match_database("ecoinvent-3.10-biosphere", fields=["name", "categories"])
i.statistics()

In [ ]:
import json
with open("ecoinvent-3.9.1-biosphere-ecoinvent-3.10-biosphere.json") as f_in:
    migration = json.load(f_in)

In [ ]:
migration = {
    (
        a["source"]["name"],
        #a["source"]["reference product"],
        #a["source"]["location"],
        #a["source"]["unit"]
    ): a["target"]
    for a in migration["replace"]
}
len(migration)

In [ ]:
for act in i.data:
    for exc in act["exchanges"]:
        if exc["type"] == "biosphere":
            key = (exc["name"],)
            if key in migration:
                if "name" in migration[key]:
                    exc["name"] = migration[key]["name"]
                exc["input"] = ("ecoinvent-3.10-biosphere", migration[key]["uuid"])

In [ ]:
i.match_database("ecoinvent-3.10-biosphere", fields=["name", "categories"])
i.statistics()

In [ ]:
i.drop_unlinked(i_am_reckless=True)
if i.db_name in bw2data.databases:
    del bw2data.databases[i.db_name]
i.write_database()

In [ ]:
bw2io.export.write_lci_excel("ESA LCA External 1.2.0f e3.9.1")

In [5]:
import bw2io
from pathlib import Path
import bw2data
from brightpath import SimaproConverter
fp="lci-ESA-LCA-External-120f-ei310.xlsx"
bw2data.projects.set_current("ecoinvent-3.11-cutoff")

In [6]:
i = bw2io.ExcelImporter(fp)
i.apply_strategies()
i.match_database(fields=["name", "reference product", "location"])
i.match_database("ecoinvent-3.11-cutoff", fields=["name", "reference product", "location"])
i.match_database("biosphere3", fields=["name", "categories"])
i.statistics()

Extracted 1 worksheets in 1.41 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 0.51 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Graph statistics for `ESA LCA External 1.2.0

(770, 29331, 469, 0)

In [7]:
import json
with open("ecoinvent-3.10.1-biosphere-ecoinvent-3.11-biosphere.json") as f_in:
    migration = json.load(f_in)

In [8]:
migration = {
    (
        a["source"]["name"],
        #a["source"]["reference product"],
        #a["source"]["location"],
        #a["source"]["unit"]
    ): a["target"]
    for a in migration["replace"]
}
len(migration)

12

In [9]:
for act in i.data:
    for exc in act["exchanges"]:
        if exc["type"] == "biosphere":
            key = (exc["name"],)
            if key in migration:
                if "name" in migration[key]:
                    exc["name"] = migration[key]["name"]
                exc["input"] = ("biosphere3", migration[key]["uuid"])

In [10]:
i.match_database("biosphere3", fields=["name", "categories"])
i.statistics()

Applying strategy: link_iterable_by_fields
Graph statistics for `ESA LCA External 1.2.0f e3.9.1` importer:
770 graph nodes:
	process: 770
29331 graph edges:
	biosphere: 22930
	technosphere: 5631
	production: 770
28913 edges to the following databases:
	biosphere3: 22930
	ecoinvent-3.11-cutoff: 3384
	ESA LCA External 1.2.0f e3.9.1: 2599
42 unique unlinked edges (418 total):
	technosphere: 42




(770, 29331, 418, 0)

In [11]:
import json
with open("ecoinvent-3.10.1-cutoff-ecoinvent-3.11-cutoff.json") as f_in:
    migration = json.load(f_in)

In [12]:
migration = {
    (
        a["source"]["name"],
        a["source"]["reference product"],
        a["source"]["location"],
        #a["source"]["unit"]
    ): a["target"]
    for a in migration["replace"]
}

In [13]:
for act in i.data:
    for exc in act["exchanges"]:
        if exc["type"] == "technosphere":
            key = (
                exc["name"],
                exc["reference product"],
                exc["location"],
                #exc["unit"]
            )
            if key in migration:
                exc["name"] = migration[key]["name"]
                exc["reference product"] = migration[key]["reference product"]
                #exc["unit"] = migration[key]["unit"]
                exc["location"] = migration[key]["location"]

In [14]:
#i.match_database(fields=["name", "reference product", "location"])
i.match_database("ecoinvent-3.11-cutoff", fields=["name", "reference product", "location"])
#i.match_database("ecoinvent-3.10-biosphere", fields=["name", "categories"])
i.statistics()

Applying strategy: link_iterable_by_fields
Graph statistics for `ESA LCA External 1.2.0f e3.9.1` importer:
770 graph nodes:
	process: 770
29331 graph edges:
	biosphere: 22930
	technosphere: 5631
	production: 770
29324 edges to the following databases:
	biosphere3: 22930
	ecoinvent-3.11-cutoff: 3795
	ESA LCA External 1.2.0f e3.9.1: 2599
5 unique unlinked edges (7 total):
	technosphere: 5




(770, 29331, 7, 0)

In [19]:
import json
with open("ecoinvent-3.10.1-cutoff-ecoinvent-3.11-cutoff.json") as f_in:
    migration = json.load(f_in)

In [20]:
migration = {
    (
        a["source"]["name"],
        a["source"]["reference product"],
        a["source"]["location"],
        #a["source"]["unit"]
    ): a["targets"]
    for a in migration["disaggregate"]
}
len(migration)

160

In [21]:
for act in i.data:
    new_excs = []
    for exc in act["exchanges"]:
        if exc["type"] == "technosphere":
            key = (
                exc["name"],
                exc["reference product"],
                exc["location"],
                #exc["unit"]
            )
            if key in migration:
                for target in migration[key]:
                    new_excs.append(
                        {
                            "name": target["name"],
                            "reference product": target["reference product"],
                            "unit": exc["unit"],
                            "location": target["location"],
                            "uncertainty type": exc.get("uncertainty type", 0),
                            "loc": exc.get("loc", 0),
                            "scale": exc.get("scale", 0),
                            "amount": exc["amount"] * target["allocation"],
                            "type": "technosphere"
                        }
                    )
                exc["delete"] = True
    if len(new_excs) > 0:
        act["exchanges"].extend(new_excs)
    act["exchanges"] = [e for e in act["exchanges"] if e.get("delete", False) is False]
                

In [22]:
#i.match_database(fields=["name", "reference product", "location"])
i.match_database("ecoinvent-3.11-cutoff", fields=["name", "reference product", "location"])
#i.match_database("ecoinvent-3.10-biosphere", fields=["name", "categories"])
i.statistics()

Applying strategy: link_iterable_by_fields
Graph statistics for `ESA LCA External 1.2.0f e3.9.1` importer:
770 graph nodes:
	process: 770
29614 graph edges:
	biosphere: 22930
	technosphere: 5914
	production: 770
29613 edges to the following databases:
	biosphere3: 22930
	ecoinvent-3.11-cutoff: 4084
	ESA LCA External 1.2.0f e3.9.1: 2599
1 unique unlinked edges (1 total):
	technosphere: 1




(770, 29614, 1, 0)

In [24]:
# 2-methylpentane to generic market for solvent, organic has been removed from ei 3.11
i.drop_unlinked(i_am_reckless=True)
if i.db_name in bw2data.databases:
    del bw2data.databases[i.db_name]
i.write_database()

Applying strategy: drop_unlinked
Applied 1 strategies in 0.03 seconds
12:50:25 [warning  ] Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|████████████████████████████████████████| 770/770 [00:01<00:00, 393.45it/s]

12:50:29 [info     ] Vacuuming database            


Created database: ESA LCA External 1.2.0f e3.9.1


In [25]:
bw2io.export.write_lci_excel("ESA LCA External 1.2.0f e3.9.1")

'/Users/romain/Library/Application Support/Brightway3/ecoinvent-311-cutoff.48ac3d3e3f6866ef5f407f0df49b277a/output/lci-ESA-LCA-External-120f-e391.xlsx'

In [38]:
import bw2io
from pathlib import Path
import bw2data
from brightpath import SimaproConverter
fp='lci-ESA-LCA-External-120f-ei391.xlsx'
#fp="lci-strathclyde_space_systems_database_1_0_3_ei310.xlsx"
bw2data.projects.set_current("ecoinvent-3.9.1-cutoff")

In [39]:
i = bw2io.ExcelImporter(fp)
i.apply_strategies()
i.match_database(fields=["name", "reference product", "location"])
i.match_database("ecoinvent-3.9.1-cutoff", fields=["name", "reference product", "location"])
i.match_database("ecoinvent-3.9.1-biosphere", fields=["name", "categories"])
i.statistics()

Extracted 1 worksheets in 1.62 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 0.43 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Graph statistics for `ESA LCA External 1.2.0

(770, 29201, 1, 0)

In [40]:
print(list(i.unlinked))

[{'name': 'Placeholder', 'amount': 2.4, 'location': 'Placeholder', 'unit': 'kilogram', 'categories': ('Materials/fuels',), 'type': 'technosphere', 'uncertainty type': 2, 'loc': 0.8754687373538999, 'scale': 0.07421000255913658, 'comment': '(2,4,3,1,2,na)Given as "Arsenic trioxide, 99%" in Mohr et al.Only included here with an empty process as a placeholder for further iteration.', 'negative': 0, 'product': 'arsenic trioxide, 99%', 'reference product': 'arsenic trioxide, 99%', 'simapro name': 'Arsenic trioxide, 99% {Placeholder} | Placeholder | Placeholder'}]


In [ ]:
from ecoinvent_migrate import *
filepath = generate_biosphere_mapping("3.9.1", "3.10")

In [ ]:
from ecoinvent_migrate import *
filepath = generate_biosphere_mapping("3.9.1", "3.10")

In [ ]:
from ecoinvent_migrate import *
filepath = generate_biosphere_mapping("3.10.1", "3.11")

In [1]:
from ecoinvent_migrate import __version__
__version__

'0.5'

In [22]:
import ecoinvent_migrate

In [ ]:
ecoinvent_migrate.

In [1]:
from ecoinvent_migrate import *
filepath = generate_technosphere_mapping("3.10.1", "3.11")

2025-01-30 12:44:17.567 | INFO     | ecoinvent_migrate.utils:configure_logs:18 - Writing logs to /Users/romain/Library/Logs/ecoinvent_migrate/2025-01-30T12-44-17
2025-01-30 12:44:18.747 | INFO     | ecoinvent_migrate.main:get_change_report_context:61 - Versions available for this license: ['3.11', '3.10.1', '3.10', '3.9.1', '3.9', '3.8', '3.7.1', '3.7', '3.6', '3.5', '3.4', '3.3', '3.2', '3.1', '3.01', '2']
2025-01-30 12:44:19.618 | INFO     | ecoinvent_migrate.main:get_change_report_context:74 - Using change report annex file Change Report Annex v3.10.1 - v3.11.xlsx
2025-01-30 12:44:24.061 | INFO     | ecoinvent_migrate.data_io:get_brightway_databases:113 - Loading source database ecoinvent-3.10.1-cutoff to cache data attributes
2025-01-30 12:44:25.124 | INFO     | ecoinvent_migrate.data_io:get_brightway_databases:118 - Loading target database ecoinvent-3.11-cutoff to cache data attributes
2025-01-30 12:44:26.506 | WARNING  | ecoinvent_migrate.wrangling:resolve_glo_row_rer_roe:221 - T